In [5]:
import yfinance as yf
import pandas as pd
import sqlite3

# -------------- ① yfinance で任天堂を取得 --------------
symbol = "7974.T"  # 任天堂
csv_path = "nintendo_2018_2022.csv"
db_path = "nintendo.db"
table_name = "nintendo"

df = yf.download(symbol, start="2018-01-01", end="2022-12-31")
df.to_csv(csv_path)

# -------------- ② CSV を読み込む --------------
df = pd.read_csv(csv_path)

print("列名チェック:", df.columns.tolist())

# -------------- ③ 日付列（Date or Unnamed: 0）を特定 --------------
if "Date" in df.columns:
    date_col = "Date"
elif "Unnamed: 0" in df.columns:
    date_col = "Unnamed: 0"
else:
    # 念のため、一番左を日付として扱う保険
    date_col = df.columns[0]

# -------------- ④ Adj Close → AdjClose にリネーム（あれば） --------------
if "Adj Close" in df.columns:
    df = df.rename(columns={"Adj Close": "AdjClose"})

# -------------- ⑤ 必要な列だけを安全に集める --------------
# まず「存在している列」をベースに needed を組み立てる
base_needed = [date_col, "Open", "High", "Low", "Close"]
optional_cols = []

if "AdjClose" in df.columns:
    optional_cols.append("AdjClose")
if "Volume" in df.columns:
    optional_cols.append("Volume")

needed = [c for c in base_needed + optional_cols if c in df.columns]

df = df[needed]

# -------------- ⑥ 日付列を 'Date' に統一 --------------
df = df.rename(columns={date_col: "Date"})

# 型も念のため文字列にしておく
df["Date"] = df["Date"].astype(str)

print("最終的な列名:", df.columns.tolist())
print(df.head())

# -------------- ⑦ SQLite に保存 --------------
conn = sqlite3.connect(db_path)
df.to_sql(table_name, conn, if_exists="replace", index=False)
conn.close()

print(f"SQLite {db_path} のテーブル {table_name} に登録完了")


C:\Users\flare\AppData\Local\Temp\ipykernel_15960\2926062993.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(symbol, start="2018-01-01", end="2022-12-31")
[*********************100%***********************]  1 of 1 completed


列名チェック: ['Price', 'Close', 'High', 'Low', 'Open', 'Volume']
最終的な列名: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
         Date              Open              High               Low  \
0      Ticker            7974.T            7974.T            7974.T   
1        Date               NaN               NaN               NaN   
2  2018-01-01  3342.91259765625  3342.91259765625  3342.91259765625   
3  2018-01-02  3342.91259765625  3342.91259765625  3342.91259765625   
4  2018-01-03  3342.91259765625  3342.91259765625  3342.91259765625   

              Close  Volume  
0            7974.T  7974.T  
1               NaN     NaN  
2  3342.91259765625       0  
3  3342.91259765625       0  
4  3342.91259765625       0  
SQLite nintendo.db のテーブル nintendo に登録完了


PS D:\cit\db2025\8> .\sqlite3.exe .\nintendo.db
SQLite version 3.39.3 2022-09-05 11:02:23
Enter ".help" for usage hints.
sqlite> .tables
nintendo

sqlite> SELECT Date, High - Low AS DailyRange
   ...> FROM nintendo
   ...> ORDER BY Date
   ...> LIMIT 20;

2018-01-01|0.0
2018-01-02|0.0
2018-01-03|0.0
2018-01-04|116.056424056841
2018-01-05|90.8973438233397
2018-01-08|0.0
2018-01-09|62.4919324935895
2018-01-10|105.505846900258
2018-01-11|35.7096729710875
2018-01-12|60.0571882178679
2018-01-15|55.1876756767583
2018-01-16|74.6656781253464
2018-01-17|66.5498549279641
2018-01-18|81.1583735389477
2018-01-19|110.375350851097
2018-01-22|88.4626237583848
2018-01-23|125.795442550791
2018-01-24|164.751454353333
2018-01-25|97.3900431919874
2018-01-26|51.1297619823795